This is the setup cell

In [1]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt

#Run this line if you want interactive plots
%matplotlib qt

import numpy as np

import sprit

In [2]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
#dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

params = sprit.input_param( acq_date='2023-02-06',
                            #starttime = obspy.UTCDateTime('2023-02-06T21:32:00.00'),
                            #endtime = obspy.UTCDateTime('2023-02-06T22:00:00.00'),
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Github/SPRIT/resources/raspshake_metadata.inv


In [3]:
params = sprit.get_metadata(params)
stream = sprit.fetch_data(params=params)

Day of Year: 37
Reading files: 
	AM.RAC84.00.EHE.D.2023.037
	AM.RAC84.00.EHN.D.2023.037
	AM.RAC84.00.EHZ.D.2023.037


c:\ProgramData\Anaconda3\envs\seismic39\lib\site-packages\obspy\core\stream.py:3197: UserWarning: No matching response information found.
  warnings.warn(str(e))


In [4]:
xwindows = sprit.select_windows(stream)

In [6]:
stream = sprit.remove_noise(stream, kind='manual', window_list=xwindows)

In [7]:
ppsdsData = sprit.generate_ppsds(params=params, stream=stream, ppsd_length=60, overlap=0.5, period_step_octaves=0.01, remove_outliers=True)

In [8]:
hvsr_dict = sprit.process_hvsr(params=ppsdsData, method=4, resample=False, smooth=10)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

No Best Peak identified


In [25]:
#Check and remove outlier curves (this often happens when time windows are removed)
def remove_outlier_curves(hvsr_dict):
    #INTENTEDED TO BE USED AS PART OF GENERATE_PPSDS function
    #RECOMMENDED TO USE OUTPUT FROM GENERATE_PPSDS!!!
    #Input can either be raw ppsd object from obpsy, output dict from generate_ppsds, or output dict from process_hvsr
    #    If raw ppsd object which case, they should be read in as a dictionary with 3 key/value pairs (1 per component))
    ppsds = hvsr_dict['ppsd_dict']
    newPPsds = {}
    stds = {}
    psds_to_rid = []

    for k in ppsds:
        psdVals = np.array(ppsds[k]['psd_values'])
        meanArr = np.nanmean(psdVals, axis=1)
        newPPsds[k] = []
        totMean = np.nanmean(meanArr)
        stds[k] = np.std(meanArr)

        for i, m in enumerate(meanArr):
            if m > totMean + 3*stds[k] or m < totMean - 3*stds[k]:
                psds_to_rid.append(i)
    psds_to_rid = np.unique(psds_to_rid)

    for k in hvsr_dict['ppsd_dict']:
        for i, r in enumerate(psds_to_rid):
            index = int(r-1)
            hvsr_dict['ppsd_dict'][k]['psd_values'] = np.delete(hvsr_dict['ppsd_dict'][k]['psd_values'], index, axis=0)
            hvsr_dict['ppsd_dict'][k]['current_times_used'] = np.delete(hvsr_dict['ppsd_dict'][k]['current_times_used'], index, axis=0)
            hvsr_dict['ppsd_dict'][k]['period_bin_centers']

    return hvsr_dict

hvsr_dict = remove_outlier_curves(hvsr_dict)

-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
-3076.527 get rid of it
(52, 901)
(52,)
(49, 901)
(49,)


In [ ]:
hvsr_dict

In [4]:
xwindows = sprit.select_windows(stream)
stream = sprit.remove_noise(stream, kind='manual', window_list=xwindows)

In [32]:
j=0
k=0
for i in hvsr_dict['psd_raw']['EHN']:
    k+=1
    if np.nanmedian(i) < -200:
        pass
    else:
        j+=1
        plt.plot((np.flip(i)))
#plt.semilogy()
plt.show()
print(j, k)

48 52


In [ ]:
period_bin_centers
psd_values
current_times_used

In [29]:
params['ppsds']['EHZ']

In [7]:
sprit.hvplot(hvsr_dict, kind='HVSR p t', show=True)

UnboundLocalError: local variable 'bestPeak' referenced before assignment

In [16]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

#fig, ax = plt.subplots()


fig, ax = sprit.hvplot(hvsr_dict=hvsr_dict, kind='spec', returnfig=True, cmap='turbo')

clickNo = 0    
xWindows = []
pathList = []
windowDrawn = []
winArtist = []
lineArtist = []

def on_click(event, fig=fig, ax=ax):
    if event.button is MouseButton.RIGHT:
        remove_on_right(event)
    if event.button is MouseButton.LEFT:
        draw_boxes(event)

def draw_boxes(event, fig=fig, ax=ax):
    global clickNo
    global x0
    global xWindows
    global pathList
    global windowDrawn
    global winArtist
    global lineArtist

    if event.inaxes!=ax: return
    #y0, y1 = ax.get_ylim()
    y0=0
    y1=50
    
    if clickNo == 0:
        #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
        x0 = event.xdata
        clickNo += 1
        linArt = plt.axvline(x0, y0, y1, linewidth=0.5, color='k', zorder=100)
        lineArtist.append([linArt, linArt])
    else:
        x1 = event.xdata
        clickNo = 0

        path_data = [
            (matplotlib.path.Path.MOVETO, (x0, y0)),
            (matplotlib.path.Path.LINETO, (x1, y0)),
            (matplotlib.path.Path.LINETO, (x1, y1)),
            (matplotlib.path.Path.LINETO, (x0, y1)),
            (matplotlib.path.Path.LINETO, (x0, y0)),
            (matplotlib.path.Path.CLOSEPOLY, (x0, y0)),
        ]

        codes, verts = zip(*path_data)
        path = matplotlib.path.Path(verts, codes)

        x_win = [x0, x1]
        xWindows.append(x_win)
        pathList.append(path)
        windowDrawn.append(False)
        winArtist.append(None)
        [lineArtist[-1].pop()]
        draw_windows(event=event, pathlist=pathList)
        linArt = plt.axvline(x1, y0, y1, color='k',linewidth=0.5, zorder=100)
        lineArtist[-1].append(linArt)
    fig.canvas.draw() 

def draw_windows(event, pathlist, fig=fig, ax=ax):
    for i, p in enumerate(pathList):
        if windowDrawn[i]:
            pass
        else:
            patch = matplotlib.patches.PathPatch(p, facecolor='k', alpha=0.75)
            winArt = ax.add_patch(patch)
            windowDrawn[i] = True
            winArtist[i] = winArt
    if event.button is MouseButton.RIGHT:
        fig.canvas.draw() 

def remove_on_right(event, fig=fig, ax=ax, xWindows=xWindows):
    if xWindows is not None:
        for i, xWins in enumerate(xWindows):
            if event.xdata > xWins[0] and event.xdata < xWins[1]:
                xWindows.pop(i)
                pathList.pop(i)
                winArtist[i].remove()
                winArtist.pop(i)
                windowDrawn.pop(i)
                lineArtist[i][0].remove()
                lineArtist[i][1].remove()
                lineArtist.pop(i)
    fig.canvas.draw() 
    #draw_windows(event, pathlist=pathList)

fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()